In [133]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.utils import np_utils
import numpy as np
import random
import sys
import io

In [143]:
path = "C:/Users/Lenovo/Desktop/gp.txt"
with io.open(path) as f:
    text = f.read().lower()
    text = text.replace('\n', ' ')
    text = text.replace('\xa0', '')
    text = text.replace('  ', ' ')
print('corpus length:', len(text))

corpus length: 112274


In [144]:
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [145]:
n_chars = len(text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  112274
Total Vocab:  84


In [147]:
text

'гарри поттер. полная коллекция джоан кэтлин роулинг  все семь электронных книг из серии о гарри поттере– которая стала международным бестселлером и отмечена множеством наград– с изумительными иллюстрациями олли мосс на обложках можно загрузить в виде одного пакета. окунитесь в историю, которая захватила воображение миллионов людей по всему миру.   дж.к. роулинг гарри поттер полная коллекция   гарри поттер и философский камень     посвящается джессике, которая любит сказки, и энн, которая тоже их любит, а еще – ди, которая первой услышала эту историю    глава первая мальчик, который остался жив  мистер и миссис дурслей, из дома №4 по бирючинной улице, гордились тем, что они, спасибо преогромное, люди абсолютно нормальные. трудно было вообразить, что они окажутся замешаны в делах необычных или загадочных – они не признавали всякой там чепухи. мистер дурслей работал директором фирмы «груннингс», которая выпускала сверла. он был большой, грузный мужчина почти без шеи, зато невероятно усат

In [148]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  112174


In [149]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [70]:
y.shape

(113957, 86)

In [150]:
# define the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [151]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [153]:
# fit the model
model.fit(X, y, epochs=12, batch_size=128, callbacks=callbacks_list)

Epoch 1/12
112174/112174 [==============================] - 163s 1ms/step - loss: 3.1966 

Epoch 00001: loss improved from inf to 3.19662, saving model to weights-improvement-01-3.1966.hdf5
Epoch 2/12
112174/112174 [==============================] - 182s 2ms/step - loss: 3.0764

Epoch 00002: loss improved from 3.19662 to 3.07640, saving model to weights-improvement-02-3.0764.hdf5
Epoch 3/12
112174/112174 [==============================] - 190s 2ms/step - loss: 3.0450

Epoch 00003: loss improved from 3.07640 to 3.04496, saving model to weights-improvement-03-3.0450.hdf5
Epoch 4/12
112174/112174 [==============================] - 201s 2ms/step - loss: 3.0291

Epoch 00004: loss improved from 3.04496 to 3.02913, saving model to weights-improvement-04-3.0291.hdf5
Epoch 5/12
112174/112174 [==============================] - 212s 2ms/step - loss: 3.0204

Epoch 00005: loss improved from 3.02913 to 3.02036, saving model to weights-improvement-05-3.0204.hdf5
Epoch 6/12
112174/112174 [============

In [154]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
filename = "weights-improvement-12-2.9676.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" миранда истреб «сборник заклинаний (часть первая)» –батильда бэгшот «история магии» –адальберт вафли "


In [128]:
max(prediction[0])

0.1561416

In [155]:
# generate characters
for i in range(100):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

  оо по по по по пе поое  оо по по по пе поое  оо по по по пе поое  оо по по по пе поое  оо по по по
Done.
